In [ ]:
# add root folder of the project to path
import sys
sys.path.insert(0, '../..')

# Load mesh & contour landmarks

In [ ]:
import pyvista as pv

mesh = pv.read('../../data/meshes/6kmh_braless_26markers/speed_6km_h_26_marker_set_.000001.obj')

In [ ]:
from mesh4d import utils

landmarks = utils.load_pkl_object('../../data/landmarks/refine_6kmh_braless_18markers_12fps.pkl')
landmarks.interp_field()

In [ ]:
contour = landmarks.extract(('marker 0', 'marker 1', 'marker 2', 'marker 3', 'marker 14', 'marker 15', 'marker 16', 'marker 17')).get_frame_coord(0)
contour_points = contour.get_points_coord()

# Estimate the contour plane

$$\begin{cases}
(\bm p_{contour} - \bm p_{center})^T \bm n = 0\\
\|\bm n\|_1 = 1
\end{cases}$$
$$\Longrightarrow \bm A \bm n = \bm b$$

Solve norm vector of the plane $\bm n$ with least-squares method. Together with the center point $p_{center}$, the (approximate) contour plane is estimated.

In [ ]:
from mesh4d.analyse import measure
contour_center = measure.points_get_center(contour_points)

In [ ]:
import numpy as np
A_up = contour_points - contour_center
A_down = np.expand_dims(
    np.ones(len(contour_center)), 
    axis=0
    )
A = np.concatenate((A_up, A_down))
A

In [ ]:
b_up = np.zeros(len(contour_points))
b_down = np.ones(1)
b = np.concatenate((b_up, b_down))
b

In [ ]:
norm_vec = np.linalg.lstsq(A, b)[0]
norm_vec

In [ ]:
norm_vec / np.linalg.norm(norm_vec)

# Mesh clipping

In [ ]:
mesh_clip = mesh.clip(norm_vec, origin=contour_center, invert=False)

In [ ]:
min_bound = measure.points_get_min_bound(contour_points)
mesh_clip = mesh_clip.clip('z', origin=min_bound, invert=False)

In [ ]:
max_bound = measure.points_get_max_bound(contour_points)
mesh_clip = mesh_clip.clip('z', origin=max_bound, invert=True)

In [ ]:
scene = pv.Plotter()
scene.add_mesh(mesh_clip, show_edges=True)
scene.camera_position = 'xy'
scene.show()